In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from keras.datasets import cifar10
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Reshape, Input, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.utils import np_utils
import csv
import keras
%matplotlib inline

Using TensorFlow backend.


In [2]:
def read_log_files(data_path, split_data=False):
    all_data = []
    for path in data_path:
        counter = 0
        log_path = path + "/driving_log.csv"
        with open(log_path) as logFile:
            reader = csv.reader(logFile)
            numOfLines = sum(1 for row in reader) - 1
            logFile.seek(1)
            for row in reader:
                if(counter > 0):
                    all_data.append(row)
                counter += 1
                    
                
    if(split_data):
        from sklearn.utils import shuffle
        #all_data = shuffle(all_data)
        val_i = np.floor(0.9*len(all_data)).astype(np.uint)
        train_data = all_data[0:val_i];
        val_data = all_data[val_i:];
        return train_data, val_data
    else:
        return all_data
    
training_path = ["../../training_data/data1", "../../training_data/data1"]
train_data, val_data = read_log_files(training_path, split_data=True)
print(len(train_data))
print(len(val_data))

swa_i = 3
cc_i = 0
lc_i=1
rc_i = 2

pr_threshold = 1

9995
1111


In [3]:
cols = 320; rows = 160

def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def trans_image(image,steer,trans_range):
    # Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    steer_ang = ((steer) + tr_x/trans_range*2*.2)
    tr_y = 40*np.random.uniform()-40/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])
    image_tr = cv2.warpAffine(image,Trans_M,(cols,rows))
    
    return image_tr,steer_ang

def add_random_shadow(image):
    top_y = 320*np.random.uniform()
    top_x = 0
    bot_x = 160
    bot_y = 320*np.random.uniform()
    image_hls = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    shadow_mask = 0*image_hls[:,:,1]
    X_m = np.mgrid[0:image.shape[0],0:image.shape[1]][0]
    Y_m = np.mgrid[0:image.shape[0],0:image.shape[1]][1]
    shadow_mask[((X_m-top_x)*(bot_y-top_y) -(bot_x - top_x)*(Y_m-top_y) >=0)]=1
    #random_bright = .25+.7*np.random.uniform()
    if np.random.randint(2)==1:
        random_bright = .5
        cond1 = shadow_mask==1
        cond0 = shadow_mask==0
        if np.random.randint(2)==1:
            image_hls[:,:,1][cond1] = image_hls[:,:,1][cond1]*random_bright
        else:
            image_hls[:,:,1][cond0] = image_hls[:,:,1][cond0]*random_bright    
    image = cv2.cvtColor(image_hls,cv2.COLOR_HLS2RGB)
    return image


def preprocess_images(X, img_length, img_width):
    img = X[40:,:,:]
    img1 = cv2.resize(img,dsize=(img_length,img_width)).astype(np.float32)
    #img1[:,:,0] -= 103.939
    #img1[:,:,1] -= 116.779
    #img1[:,:,2] -= 123.68
    return img1

def preprocess_image_file_train(line):
    i_lrc = np.random.randint(3)
    if (i_lrc == 0):
        img_path = line[lc_i][1:]
        shift_ang = .25
    if (i_lrc == 1):
        img_path = line[cc_i]
        shift_ang = .0
    if (i_lrc == 2):
        img_path = line[rc_i][1:]
        shift_ang = -.25
        
    shift_ang = .0
    swa = float(line[swa_i])+ shift_ang
    
    if(img_path[0:5] != '/home'):
        img_path = "../../training_data/data/"+img_path
    else:
        img_path = img_path
    raw_img = cv2.imread(img_path,cv2.IMREAD_COLOR)
    raw_img = cv2.cvtColor(raw_img,cv2.COLOR_BGR2RGB)
    image,y_steer = trans_image(raw_img,swa,100)
    image = augment_brightness_camera_images(image)
    image = preprocess_images(image, 64, 64)
    #image = np.array(image)
    ind_flip = np.random.randint(2)
    if ind_flip==0:
        image = cv2.flip(image,1)
        y_steer = -y_steer
        
    '''
    pr_unif = np.random.uniform()
    if(pr_unif < 0.3):
        add_random_shadow(image)
    '''
    return image,y_steer

def data_augmentation_engine(data, batch_size = 32):
    batch_images = np.zeros((batch_size, 64, 64, 3))
    batch_steering = np.zeros(batch_size)
    while True:
        for i_batch in range(batch_size):
            i_line = np.random.randint(len(data))
            keep_pr = 0 
            pr_threshold = 0.6
            while keep_pr == 0:
                x,y = preprocess_image_file_train(data[i_line])
                pr_unif = np.random
                if abs(y)<(.1):
                    pr_val = np.random.uniform()
                    if pr_val>pr_threshold:
                        keep_pr = 1
                else:
                    keep_pr = 1
                    
            batch_images[i_batch] = x
            batch_steering[i_batch] = y
        yield batch_images, batch_steering

In [4]:
def complete_model():

    img_input = Input(shape=[64,64,3])
    
    x = Convolution2D(3, 1, 1, init='he_normal', activation='relu', border_mode='same', name='norm')(img_input)

    x = Convolution2D(32, 3, 3, init='he_normal', activation='relu', border_mode='same', name='block1_conv1')(x)
    x = Convolution2D(32, 3, 3, init='he_normal', activation='relu', border_mode='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    x = Dropout(0.5)(x)

    x = Convolution2D(64, 3, 3, init='he_normal', activation='relu', border_mode='same', name='block2_conv1')(x)
    x = Convolution2D(64, 3, 3, init='he_normal', activation='relu', border_mode='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    x = Dropout(0.5)(x)

    x = Convolution2D(128, 3, 3, init='he_normal', activation='relu', border_mode='same', name='block3_conv1')(x)
    x = Convolution2D(128, 3, 3, init='he_normal', activation='relu', border_mode='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    x = Dropout(0.5)(x)

    x = Flatten(name='my_flatten_1')(x)
    x = Dense(512, init='he_normal', activation='relu', name='my_fc1')(x)
    x = Dense(64, init='he_normal', activation='relu', name='my_fc2')(x)
    x = Dense(16, init='he_normal', activation='relu', name='my_fc3')(x)
    x = Dense(1, name='predictions')(x)

    model = Model(img_input, x, name='my_model')

    return model

In [ ]:
val_images = np.zeros((len(val_data),64,64,3))
val_swa = np.zeros((len(val_data)))           
for i, line in enumerate(val_data):
    if(str(line[cc_i])[0:5] != '/home'):
        img_path = "../../training_data/data/"+str(line[cc_i])
    else:
        img_path = str(line[cc_i])
    img = cv2.imread(img_path,cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    val_images[i] = preprocess_images(img,64,64)
    val_swa[i] = float(line[3])

In [ ]:
batch_size = 512
final_model = complete_model()

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#hyperparameter_selection(final_model, adam, 30, 'learning_rate', X_val, y_val)

final_model.compile(loss='mean_squared_error',
          optimizer=adam,
          metrics=['mean_absolute_error'])

model_checkpoint = keras.callbacks.ModelCheckpoint('model.h5', monitor='loss', save_best_only=False)

for i in range(20):
    print("epoch: " + str(i))
    train_generator = data_augmentation_engine(train_data, batch_size)
    final_model.fit_generator(
            train_generator,
            samples_per_epoch=30000,
            validation_data=(val_images,val_swa),
            nb_epoch=1,
            callbacks=[model_checkpoint])

    swa = final_model.predict(val_images,512)

    plt.plot(val_swa)
    plt.plot(swa,'r')
    plt.show()

final_model.save('model.h5')
json_string = final_model.to_json()
text_file = open("model.json", "w")
text_file.write(json_string)
text_file.close()
final_model.summary()

In [ ]:
batch_size = 512
final_model = complete_model()
final_model.summary()
final_model.load_weights('model.h5', by_name=True)

swa = final_model.predict(val_images,512)

plt.plot(val_swa)
plt.plot(swa,'r')
plt.show()